In [14]:
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
import logging
import pyodbc
import pandas
import os
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexerClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SearchIndexerDataContainer, SearchIndexerDataSourceConnection)
from azure.search.documents.indexes.models import (
    SearchField, SearchFieldDataType, 
    VectorSearch, HnswAlgorithmConfiguration, VectorSearchProfile, AzureOpenAIVectorizer, AzureOpenAIVectorizerParameters,
    SearchIndex, HnswParameters, VectorSearchAlgorithmMetric, SemanticConfiguration, SemanticPrioritizedFields, 
    SemanticField, SemanticSearch, ExhaustiveKnnAlgorithmConfiguration, ExhaustiveKnnParameters,FieldMapping)
from azure.search.documents.indexes.models import (  
SplitSkill, InputFieldMappingEntry, OutputFieldMappingEntry,AzureOpenAIEmbeddingSkill, SearchIndexerIndexProjection, SearchIndexerIndexProjectionSelector,
SearchIndexerIndexProjectionsParameters,SearchIndexerSkill,SearchIndexerSkillset,SearchIndexer
)

In [15]:
load_dotenv(override=True)

sql_server = os.environ.get("SQL_SERVER_NAME")
database_name = os.environ.get("SQL_DATABASE_NAME")
username = os.environ.get("SQL_USERNAME")
password = os.environ.get("SQL_PASSWORD")

In [16]:
azure_openai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
azure_openai_gpt_deployment = os.getenv("AZURE_OPENAI_GPT_DEPLOYMENT", "gpt-4.1-mini")
azure_openai_gpt_model = os.getenv("AZURE_OPENAI_GPT_MODEL", "gpt-4.1-mini")
azure_openai_embedding_deployment = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT", "text-embedding-3-large")
azure_openai_embedding_model = os.getenv("AZURE_OPENAI_EMBEDDING_MODEL", "text-embedding-3-large")

In [17]:
endpoint = os.environ["AZURE_SEARCH_ENDPOINT"]
# credential = DefaultAzureCredential(managed_identity_client_id  = "4367ceb1-6348-41d1-92a1-b5583f2ec5de")
credential = AzureKeyCredential("Z9qrhCZDgy2NYcqxdaIAzbEZ8IeSrNnsQluoogM6N0AzSeD1Fr3z")

In [18]:
embedding_length = 3072
table_name = '[SalesLT].[Customer]'
data_source_name = f"{table_name.replace('[','').replace(']','').replace('.','').lower()}-azuresqlcon"
data_source_name


'salesltcustomer-azuresqlcon'

In [19]:
sqltcpcon = f'Encrypt=True;TrustServerCertificate=False;Connection Timeout=30;Server=tcp:{sql_server};Database={database_name};User ID={username};Password={password};'

aisearch = SearchIndexerClient(endpoint, credential)
search_container = SearchIndexerDataContainer(name=table_name)

data_source_connection = SearchIndexerDataSourceConnection(
        name=data_source_name,
        type="azuresql",
        connection_string=sqltcpcon,
        container=search_container
    )

datacon = aisearch.create_or_update_data_source_connection(data_source_connection)
logging.info(f"Data source connection {datacon.name} created")

In [20]:
index_name = 'sql-db-index'
skillset_name = index_name + "-skillset"
indexer_name = f"{index_name}-indexer"

In [21]:
index = SearchIndex(
    name=index_name,
        fields=[
            SearchField(name="Id", type=SearchFieldDataType.String, key=True, analyzer_name="keyword"),
            SearchField(name="chunk", type=SearchFieldDataType.String, sortable=False, filterable=False, facetable=False),
            SearchField(name="vector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single), vector_search_dimensions=embedding_length, vector_search_profile_name="vectorsearch-profile"),
            SearchField(name="db_CustomerID", type=SearchFieldDataType.String, sortable=True, filterable=True, facetable=True),
            SearchField(name="db_FirstName", type=SearchFieldDataType.String, sortable=True, filterable=True, facetable=True),
            SearchField(name="db_CompanyName", type=SearchFieldDataType.String, sortable=True, filterable=True, facetable=True)
    ],
    vector_search=VectorSearch(
        profiles=[ 
            VectorSearchProfile(
                name="vectorsearch-profile",
                algorithm_configuration_name="hnsw-config",
                vectorizer_name="azure_openai_text_3_large" 
            ),
            VectorSearchProfile(
                name="exhaustiveknn-profile",
                algorithm_configuration_name="exhaustiveknn-config",
                vectorizer_name="azure_openai_text_3_large"
            )
        ],
        algorithms=[
            HnswAlgorithmConfiguration(
                name="hnsw-config",
                parameters=HnswParameters(  
                    m=4,  
                    ef_construction=400,  
                    ef_search=500,  
                    metric=VectorSearchAlgorithmMetric.COSINE,  
                ),  
            ),
            ExhaustiveKnnAlgorithmConfiguration(
                name="exhaustiveknn-config",
                parameters=ExhaustiveKnnParameters(
                    metric=VectorSearchAlgorithmMetric.COSINE
                )
            )
        ],
        vectorizers=[
            AzureOpenAIVectorizer(
                vectorizer_name="azure_openai_text_3_large",
                kind="azureOpenAI",
                parameters=AzureOpenAIVectorizerParameters(
                    resource_url=azure_openai_endpoint,
                    deployment_name=azure_openai_embedding_deployment,
                    model_name=azure_openai_embedding_model
                )
            )
        ]
    ),
    semantic_search=SemanticSearch(
        default_configuration_name="semantic_config",
        configurations=[
            SemanticConfiguration(
                name="semantic_config",
                prioritized_fields=SemanticPrioritizedFields(
                    content_fields=[
                        SemanticField(field_name="chunk")
                    ]
                )
            )
        ]
    )
)

index_client = SearchIndexClient(endpoint=endpoint, credential=credential)
index_client.create_or_update_index(index)
print(f"Index '{index_name}' created or updated successfully")

Index 'sql-db-index' created or updated successfully


In [22]:
split_skill = SplitSkill(
        description="Split skill to chunk documents",
        text_split_mode="pages",
        context="/document",
        maximum_page_length=300,
        page_overlap_length=20,
        inputs=[
            InputFieldMappingEntry(name="text", source="/document/CompanyName")
        ],
        outputs=[
            OutputFieldMappingEntry(name="textItems", target_name="pages")
        ]    
    )

embedding_skill = AzureOpenAIEmbeddingSkill(
        description="Skill to generate embeddings via Azure OpenAI",  
        context="/document/pages/*", 
        resource_url=azure_openai_endpoint,
        deployment_name=azure_openai_embedding_deployment,
        model_name=azure_openai_embedding_model,
        inputs=[
            InputFieldMappingEntry(name="text", source="/document/pages/*")
        ],
        outputs=[
            OutputFieldMappingEntry(name="embedding", target_name="vector")
        ]
    )

index_projections = SearchIndexerIndexProjection(
        selectors=[
            SearchIndexerIndexProjectionSelector(
                target_index_name=index_name,
                parent_key_field_name="db_CustomerID",
                source_context="/document/pages/*",
                mappings=[
                    InputFieldMappingEntry(name="chunk", source="/document/pages/*"),
                    InputFieldMappingEntry(name="vector", source="/document/pages/*/vector"),
                    InputFieldMappingEntry(name="db_FirstName", source="/document/FirstName"),
                    InputFieldMappingEntry(name="db_CompanyName", source="/document/CompanyName")                    
                ]
            )
        ]
    )

skillset = SearchIndexerSkillset(
        name=skillset_name,
        description="Skillset for Azure AI Search with Azure OpenAI Embedding",
        skills=[
            split_skill,
            embedding_skill
        ],
        index_projection=index_projections
    )
c = SearchIndexerClient(endpoint, credential)  
c.create_or_update_skillset(skillset) 

In [23]:
indexer = SearchIndexer(
        name=indexer_name,
        description="Indexer to index data from Azure SQL DB, chunk text and vectorize it",
        skillset_name=index_name + "-skillset",
        target_index_name=index_name,
        data_source_name=data_source_name,
        field_mappings=[
            FieldMapping(
                    source_field_name="CustomerID",
                    target_field_name="Id",
                )]
    )

indexer_c = SearchIndexerClient(endpoint, credential)
indexer_result = indexer_c.create_or_update_indexer(indexer)


In [25]:
indexer_c.run_indexer(indexer_name) #add waiting time for bigger dbs, insert status check
logging.info(f"Running indexer {indexer_name} is finished")

In [ ]:
# Check if the index exists and get its status
from azure.search.documents import SearchClient

# Get index information
try:
    index_info = index_client.get_index(index_name)
    print(f"✅ Index '{index_name}' exists!")
    print(f"   Fields: {len(index_info.fields)} fields")
    print(f"   Vector search enabled: {index_info.vector_search is not None}")
    print(f"   Semantic search enabled: {index_info.semantic_search is not None}")
except Exception as e:
    print(f"❌ Index not found: {e}")

# Check indexer status
try:
    indexer_status = indexer_c.get_indexer_status(indexer_name)
    print(f"\n📊 Indexer Status:")
    print(f"   Status: {indexer_status.status}")
    print(f"   Last result: {indexer_status.last_result.status if indexer_status.last_result else 'None'}")
    if indexer_status.last_result:
        print(f"   Items processed: {indexer_status.last_result.item_count}")
        print(f"   Errors: {indexer_status.last_result.error_count}")
except Exception as e:
    print(f"❌ Indexer status error: {e}")

# Query the index to see some data
try:
    search_client = SearchClient(endpoint, index_name, credential)
    results = search_client.search("*", top=3)
    
    print(f"\n🔍 Sample data from index:")
    for i, result in enumerate(results, 1):
        print(f"   Document {i}:")
        print(f"     ID: {result.get('Id', 'N/A')}")
        print(f"     Customer: {result.get('db_FirstName', 'N/A')}")
        print(f"     Company: {result.get('db_CompanyName', 'N/A')}")
        print(f"     Chunk: {result.get('chunk', 'N/A')[:50]}...")
        print()
        
except Exception as e:
    print(f"❌ Query error: {e}")

print(f"\n🌐 You can also view this index at:")
print(f"   Azure Portal: {endpoint.replace('https://', 'https://portal.azure.com/#@/resource')}")
print(f"   Index name: {index_name}")
print(f"   Direct URL: {endpoint}/indexes/{index_name}")